In [35]:
import sys
import os

print(sys.version)

3.12.3 (main, May  7 2024, 08:28:12) [GCC 9.4.0]


In [36]:
# tmp 디렉토리가 없으면 생성합니다.
if not os.path.isdir('tmp'):
    os.mkdir('tmp')
    # Abalone Dataset을 다운로드 받습니다.
    !wget https://archive.ics.uci.edu/static/public/1/abalone.zip -P tmp
    !unzip tmp/abalone.zip

In [37]:
# Abalone 데이터셋으 설명이 들어간 파일을 출력합니다.
!cat tmp/abalone.names

1. Title of Database: Abalone data

2. Sources:

   (a) Original owners of database:
	Marine Resources Division
	Marine Research Laboratories - Taroona
	Department of Primary Industry and Fisheries, Tasmania
	GPO Box 619F, Hobart, Tasmania 7001, Australia
	(contact: Warwick Nash +61 02 277277, wnash@dpi.tas.gov.au)

   (b) Donor of database:
	Sam Waugh (Sam.Waugh@cs.utas.edu.au)
	Department of Computer Science, University of Tasmania
	GPO Box 252C, Hobart, Tasmania 7001, Australia

   (c) Date received: December 1995


3. Past Usage:

   Sam Waugh (1995) "Extending and benchmarking Cascade-Correlation", PhD
   thesis, Computer Science Department, University of Tasmania.

   -- Test set performance (final 1044 examples, first 3133 used for training):
	24.86% Cascade-Correlation (no hidden nodes)
	26.25% Cascade-Correlation (5 hidden nodes)
	21.5%  C4.5
	 0.0%  Linear Discriminate Analysis
	 3.57% k=5 Nearest Neighbour
      (Problem encoded as a classification task)

   -- Data set samp

# 분석의 목표

Rings를 타겟으로 하는 머신러닝 모델을 만들어 Rings를 예측하는 모델을 만듭니다.

모델의 성능 지표는 RMSLE로 합니다. 

$\sqrt{ \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2}$

Kaggle에서도 이 지표를 이용한 Playground 에서 RMSLE를 사용하여 다루었던 주제입니다.

[Kaggle: Regression with an Abalone Dataset](https://www.kaggle.com/competitions/playground-series-s4e4)

여기서는 UCI Repository에서의 데이터만을 가지고, Machine Learning 모델을 만들어 봅니다.

## 실험의 설정

UCI의 abalone 데이터셋의 70%는 학습용, 30%는 평가용 데이터셋으로 만듭니다. 이 때 Rings의 비율이 동일하도록 합니다.

## 노트에서 보일 sgml의 내용

1. 변수의 타입 최적화 합니다.

2. 변수의 처리 내역 및 특성을 데이터프레임으로 관리합니다.

3. 변수의 처리를 위한 탐색적 과정들은 생략합니다.

4. 학습데이터와 평가데이터에 적용할 처리 과정을 list로 정의하여 일괼 처리할 수 있게 합니다.

5. pl.DataFrame에서는 전처리단계에서 사용하고 pd.DataFrame은 Machinne Learning 단계에서 사용합니다. 즉,  pl.DataFrame을 통해 전처리가 끝나면, pd.DataFrame으로 사용합니다.

- 즉, Polars의 데이터 처리의 효율화를 취하고, pd.DataFrame의 타 모듈과 호환성의 장점을 취합니다.

6. sgml에 있는 Tabular 데이터에 특화된 Neural Network 모델을 활용해봅니다.

7. sgml의 SGStacking을 통해 실험 결과를 관리하고 이를 종합하여 stacking 모델을 만듭니다.


## 데이터 처리과정

1. 변수 스펙이 담긴 abalone.names 파일을 기반으로 변수 스펙을 지닌 데이터프레임(df_feature)를 정의합니다. 이 데이터프레임을 통해 변수의 처리 사항이나 특징을 관리합니다.

  - Index는 변수명, Description는 변수에 대한 설명, Unit은 단위, org는 변수의 출처를 나타냅니다.

2. 데이터에 적합한 타입을 파악하기 위해 pl.read_csv → dproc.get_type_df 를 사용하여 변수의 범위와 범주의 수를 파악합니다.

3. dproc.get_type_pl을 이용하여 적합한 변수 타입을 도출합니다.

4. 파악한 변수 타입으로 데이터를 불러 옵니다.

5. functools.partial을 이용하여 dproc의 데이터 처리 함수들에 파라미터를 설정하여 pl.DataFrame을 전달만 하도록 설정 method chaining이 가능한 형태로 데이터를 처리합니다.

- Rings를 log(Rings + 1) 변환을 하여 target 변수를 만듭니다.

- Height 변수에서 이상치들을 범위를 한정 시킵니다.

- 면적(square), 부피(volumn), 밀도(density)를 추가합니다.

- 연속형 입력 변수들 표준화하고 PCA 주성분 4개를 만듭니다.


In [38]:
import dproc
import pandas as pd
import polars as pl
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

for i in [pd, pl, np, mpl, sns]:
    print(i.__name__, i.__version__)

pandas 2.2.2
polars 0.20.24
numpy 1.26.4
matplotlib 3.8.4
seaborn 0.13.2


In [39]:
from functools import partial

# pandas DataFrame에서 텍스트 표시할 때 텍스트 줄임을 하지 않는 최대의 텍스트 길이를 지정합니다.
pd.set_option('display.max_colwidth', 255)

In [40]:
# 변수들에 대한 설명을 담은 데이터프레임입니다. 
# Index는 변수명, Description은 변수에 대한 설명을 나타냅니다. 
df_feature = pd.DataFrame({
    "Description" : [
            "M, F, and I (infant)",
            "Longest shell measurement",
            "perpendicular to length",
            "with meat in shell",
            "whole abalone",
            "weight of meat",
            "gut weight (after bleeding)",
            "after being dried",
            "+1.5 gives the age in years"], 
    "Units": ['', 'mm', 'mm', 'mm', 'grams', 'grams', 'grams', 'grams', '']
    }, index = ['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings']
)
df_feature

,Description,Units
Sex,"M, F, and I (infant)",
Length,Longest shell measurement,mm
Diameter,perpendicular to length,mm
Height,with meat in shell,mm
Whole weight,whole abalone,grams
Shucked weight,weight of meat,grams
Viscera weight,gut weight (after bleeding),grams
Shell weight,after being dried,grams
Rings,+1.5 gives the age in years,


In [41]:
# Abalone 데이터셋에 적합한 데이터 타입을 찾기 위한 정보를 가져옵니다.
df_type = pl.read_csv('tmp/abalone.data', has_header=False, new_columns=df_feature.index.tolist()).pipe(
    dproc.get_type_df
)
df_type

,min,max,na,count,n_unique,dtype,f32,i32,i16,i8
feature,,,,,,,,,,
Sex,NaN,NaN,0.0,4177.0,3.0,String,NaN,NaN,NaN,NaN
Length,0.0750,0.8150,0.0,4177.0,134.0,Float64,True,True,True,True
Diameter,0.0550,0.6500,0.0,4177.0,111.0,Float64,True,True,True,True
Height,0.0000,1.1300,0.0,4177.0,51.0,Float64,True,True,True,True
Whole weight,0.0020,2.8255,0.0,4177.0,2429.0,Float64,True,True,True,True
Shucked weight,0.0010,1.4880,0.0,4177.0,1515.0,Float64,True,True,True,True
Viscera weight,0.0005,0.7600,0.0,4177.0,880.0,Float64,True,True,True,True
Shell weight,0.0015,1.0050,0.0,4177.0,926.0,Float64,True,True,True,True
Rings,1.0000,29.0000,0.0,4177.0,28.0,Int64,True,True,True,True


In [42]:
# 병렬화 기능을 가진 polars로 전처리를 합니다.
# Polars를 불러 올 때 사용하기 위한 데이터 타입을 가져옵니다.
pl_dtypes = dproc.get_type_pl(df_type)
pl_dtypes

{'Length': Float32,
 'Diameter': Float32,
 'Height': Float32,
 'Whole weight': Float32,
 'Shucked weight': Float32,
 'Viscera weight': Float32,
 'Shell weight': Float32,
 'Rings': Int8,
 'Sex': Categorical}

In [43]:
# dfl_로 시작하면 pl.DataFrame
dfl_abalone = pl.read_csv('tmp/abalone.data', has_header=False, new_columns=df_feature.index.tolist(), dtypes=pl_dtypes)
dfl_abalone

Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
cat,f32,f32,f32,f32,f32,f32,f32,i8
"""M""",0.455,0.365,0.095,0.514,0.2245,0.101,0.15,15
"""M""",0.35,0.265,0.09,0.2255,0.0995,0.0485,0.07,7
"""F""",0.53,0.42,0.135,0.677,0.2565,0.1415,0.21,9
"""M""",0.44,0.365,0.125,0.516,0.2155,0.114,0.155,10
"""I""",0.33,0.255,0.08,0.205,0.0895,0.0395,0.055,7
…,…,…,…,…,…,…,…,…
"""F""",0.565,0.45,0.165,0.887,0.37,0.239,0.249,11
"""M""",0.59,0.44,0.135,0.966,0.439,0.2145,0.2605,10
"""M""",0.6,0.475,0.205,1.176,0.5255,0.2875,0.308,9


In [44]:
# 불러운 데이터 타입을 feature 데이터프레임에 추가 합니다.
# src 컬럼에는 데이터의 출처를 나타내고,
# 원래 데이터가 포함하고 있는 변수들은 org로 나타냅니다.
df_feature = df_feature.join(pd.Series(pl_dtypes, name='type').apply(lambda x: str(x))).assign(
    src = 'org'
)
df_feature

,Description,Units,type,src
Sex,"M, F, and I (infant)",,Categorical,org
Length,Longest shell measurement,mm,Float32,org
Diameter,perpendicular to length,mm,Float32,org
Height,with meat in shell,mm,Float32,org
Whole weight,whole abalone,grams,Float32,org
Shucked weight,weight of meat,grams,Float32,org
Viscera weight,gut weight (after bleeding),grams,Float32,org
Shell weight,after being dried,grams,Float32,org
Rings,+1.5 gives the age in years,,Int8,org


In [45]:
# pl.DataFrame은 sklearn.model_selection train_test_split가 지원하지 않아 수동으로 학습과 평가 데이터를 나눕니다. 
# 0으로 시작하는 데이터의 인덱스 컬럼 no 를 만들고 Rings로 구분하여 no 리스트를 만들고,
# np.random.choice를 이용하여 인덱스를 섞어 줍니다. 
idx = [
    np.random.choice(i, size=len(i), replace=False)
    for i in dfl_abalone.with_columns(pl.int_range(pl.len()).alias('no')).group_by('Rings').agg(pl.col('no'))['no']
]
# 70%는 train의 인덱스로 가져와 모으고, 30%는 평가셋 인덱스로 모읍니다.
# 처음부터 위치 인덱스가 70%인 지점까지 학습셋의 인덱스로 사용합니다.
train_idx = np.hstack([i[:int((np.ceil(len(i) * 0.7)))] for i in idx])
# 이후의 인덱스들은 평가셋으로 사용합니다.
test_idx = np.hstack([i[int((np.ceil(len(i) * 0.7))):] for i in idx])
# 추출한 인덱스로 행들을 선택합니다.
dfl_train = dfl_abalone[train_idx] 
dfl_test = dfl_abalone[test_idx]

In [46]:
len(dfl_train), len(dfl_test)

(2937, 1240)

In [47]:
# target: log(Rings + 1)이 dfl_train에 생성됩니다.
display(dfl_train.head())
df_feature

Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
cat,f32,f32,f32,f32,f32,f32,f32,i8
"""F""",0.655,0.53,0.19,1.428,0.493,0.318,0.565,18
"""M""",0.64,0.505,0.165,1.4435,0.6145,0.3035,0.39,18
"""M""",0.525,0.425,0.12,0.8665,0.2825,0.176,0.29,18
"""M""",0.645,0.485,0.155,1.489,0.5915,0.312,0.38,18
"""F""",0.645,0.49,0.19,1.3065,0.479,0.3565,0.345,18


,Description,Units,type,src
Sex,"M, F, and I (infant)",,Categorical,org
Length,Longest shell measurement,mm,Float32,org
Diameter,perpendicular to length,mm,Float32,org
Height,with meat in shell,mm,Float32,org
Whole weight,whole abalone,grams,Float32,org
Shucked weight,weight of meat,grams,Float32,org
Viscera weight,gut weight (after bleeding),grams,Float32,org
Shell weight,after being dried,grams,Float32,org
Rings,+1.5 gives the age in years,,Int8,org


In [48]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

# Height 의 변수폭을 0.01, 0.025로 한정시킵니다. 이 기준을 도출하기 위한 과정은 생략합니다.
procs = list()
procs.append(
    partial(dproc.with_columns_opr,
        proc_list=[
            ('targetproc', 'target', (pl.col('Rings') + 1).log().cast(pl.Float32), 'RMSLE 지표를 RMSE로 다루어서 분석과정을 보다 간단하게 하기 위한 log1p 변환을 하여 target을 만듭니다.'),
            ('clip_height', 'Height_n', pl.col('Height').clip(0.01, 0.025),  'Height에 이상치로 보이는 값이 있어, 0.01과 0.025 사이의 값으로 한정합니다.')
        ]
    )
)

# with_columns_opr로도 충분히 구현을 할 수 있지만, select_opr을 예시하기 위한 과정을 보입니다.
# 면적, 부피, 밀도 변수를 넣습니다.
procs.append(partial(
    dproc.select_opr, 
    select_proc = lambda x: x.select(
        pl.col('Whole weight'), 
        square = pl.col('Length') * pl.col('Diameter'), 
        volumn = pl.col('Length') * pl.col('Diameter') * pl.col('Height_n')
    ).select(pl.col('square'), pl.col('volumn'), density=pl.col('volumn') / pl.col('Whole weight')),
    desc = [
        ('square', 'Length × Diameter'),
        ('volumn', 'Length × Diameter × Height_n'),
        ('density', 'volumn ÷ Whole Weighht')
    ]
))

# 연속형 입력 변수들을 표준화하고, 표준화된 변수를 주성분을 4개로 뽑도록 PCA 변환을 포함합니다. 
X_std= ['Length', 'Diameter', 'Height_n', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'square', 'volumn', 'density']
X_pca = ['Length', 'Diameter', 'Height_n', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight']
std_pca = make_pipeline(StandardScaler(), ColumnTransformer([
    ('std', 'passthrough', np.arange(len(X_std))),
    ('pca', PCA(n_components=4), np.arange(len(X_pca)))
]))

# std_pca의 get_feature_names_out()를 통해 처리 후 변수명을 얻을 수 있습니다.
# 이는 {전처리명, 위에서는 std, pca}__{변수명}으로 되어 있습니다.
# 이렇게 전처리명과 변수명 두 개의 항목으로 std_pca에 변수명이 구성되어 있다면, 
# dproc.apply_processor에서는 p에는 전처리명을 v에는 변수명을 넘겨 줍니다.
# info_prov에 전달하는 함수명을 이를 기반으로 변수의 출처정보(org), 설명(Description), 변수 타입을 전달해줍니다.
def std_pca_prov(p, v):
    if p == 'pca':
        # PCA 변환일 경우는 org는 pca, 설명에는 Size Features PCA components v 이고 변수 타입은 pl.Float32로 합니다.
        return ('pca', v, 'Size features PCA component ' + v, pl.Float32)
    return ('std', v, 'StandardScaler: ' + v, pl.Float32)

procs.append(
    partial(dproc.apply_processor, processor = std_pca, X_val = X_std, info_prov = std_pca_prov)
)

ord_enc = OrdinalEncoder(dtype=np.int16, handle_unknown='use_encoded_value', unknown_value=-1)
procs.append(
    partial(dproc.apply_processor, processor = ord_enc, X_val = ['Sex'], info_prov=dproc.ord_prov)
)

In [49]:
# procs할 때 df_feat에 df_feature를 전달하면, processor들은 fit을 하게 되고, 
# 반환되는 df_feature에는 procs에 정의된 변수에 대한 사항이 들어가 있게 됩니다.
dfl_train, df_feature = dproc.apply_procs(dfl_train, procs, df_feat=df_feature)
# procs를 적용할 때 df_feature를 전달하지 않으면, processor들은 transform만 합니다.
dfl_test, _  = dproc.apply_procs(dfl_test, procs)

In [50]:
dfl_train

Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,target,Height_n,square,volumn,density,pca0,pca1,pca2,pca3
i16,f32,f32,f32,f32,f32,f32,f32,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,1.094114,1.228894,0.19,1.226734,0.605874,1.251694,2.350209,18,2.944439,0.041794,1.291903,1.292016,-0.704645,3.162969,0.068637,-0.101341,1.269281
2,0.96909,0.977511,0.165,1.258423,1.155085,1.119545,1.089222,18,2.944439,0.041794,1.037857,1.038137,-0.86874,2.683105,0.037703,0.211239,0.01454
2,0.010571,0.173084,0.12,0.078787,-0.345638,-0.04246,0.368659,18,2.944439,0.041794,-0.023673,-0.022695,-0.581594,0.100174,-0.037586,-0.209092,0.468003
2,1.010765,0.776405,0.155,1.351444,1.051119,1.197012,1.017166,18,2.944439,0.041794,0.927806,0.928158,-0.986752,2.617269,0.038897,0.309018,0.01783
0,1.010765,0.826681,0.19,0.978336,0.542591,1.602574,0.764969,18,2.944439,0.041794,0.962015,0.962345,-0.714869,2.339829,0.020036,0.098706,-0.013344
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,0.218945,0.072531,0.135,-0.108278,-0.257494,-0.015118,0.15249,23,3.178054,0.041794,0.030689,0.031632,-0.265278,0.025799,-0.049072,-0.290539,0.201166
0,0.802391,1.128341,0.225,0.726871,0.086046,0.814234,1.125251,23,3.178054,0.041794,1.029371,1.029657,-0.45417,1.912067,0.00073,-0.469807,0.585697
2,0.885741,0.776405,0.175,0.965047,0.336919,1.037521,1.593617,23,3.178054,0.041794,0.850638,0.851041,-0.773556,2.28311,0.036614,-0.039869,0.865682


In [51]:
# df_feature의 처리 사항을 종합합니다.
df_feature = pd.concat([
    df_feature.groupby(df_feature.index)['Description'].apply(lambda x: ','.join(x)), 
    df_feature.groupby(df_feature.index)['type'].last(),
    df_feature.groupby(df_feature.index)['src'].apply(lambda x: '→'.join(x)),
    df_feature.groupby(df_feature.index)[[i for i in df_feature.columns if i not in ['Description', 'type', 'src']]].last()
], axis=1).sort_index().sort_values('type')
df_feature

,Description,type,src,Units
Diameter,"perpendicular to length,StandardScaler: Diameter",Float32,org→std,mm
square,"Length × Diameter,StandardScaler: square",Float32,square→std,None
pca3,Size features PCA component pca3,Float32,pca,None
pca2,Size features PCA component pca2,Float32,pca,None
pca1,Size features PCA component pca1,Float32,pca,None
pca0,Size features PCA component pca0,Float32,pca,None
density,"volumn ÷ Whole Weighht,StandardScaler: density",Float32,density→std,None
Whole weight,"whole abalone,StandardScaler: Whole weight",Float32,org→std,grams
Viscera weight,"gut weight (after bleeding),StandardScaler: Viscera weight",Float32,org→std,grams
Shucked weight,"weight of meat,StandardScaler: Shucked weight",Float32,org→std,grams


In [52]:
# pd.DataFrame을 사용합니다.
df_train = dfl_train.to_pandas()
del dfl_train
df_test = dfl_test.to_pandas()
del dfl_test